# Tutorial 3 - KNN and SVR: Hyper-parameter tuning and model testing
In this third tutorial, we will look at two Machine Learning algorithms, K-Nearest Neighbours (KNN) and Support Vector Regression (SVR). For these algorithms, we will look at three aspects:
1. **Effect of hyper-parameters**: First, we will look into the effect of different hyper-parameters on the model performance, especially with regards to over-fitting and under-fitting.
2. **Hyper-parameter tuning using cross-validation**: We will look at some systematic approaches to optimize the hyper-parameters of KNN and SVR. 
3. **Model testing**: After hyper-parameter tuning, we should always test the model's performance on an independent test set *which was not used for tuning*. The error metrics obtained from the test set are used to report a model's performance and to compare different models.

# Part 1 - Load and preprocess data
Both the KNN and the SVR algorithm perform best if their features are scaled. We can hence load the scaled and shuffled dataset we have created in Tutorial 2, as it contains all pre-processing steps we have previously performed. During preprocessing, we will also split the data into features and targets, select the k best features (see Tutorial 2) and divide into training and validation set.

## Step 1.1: Import packages and data

As before, we shall always start by loading necessary packages:

In [ ]:
# Import the necessary packages to read the data
import pandas as pd
import numpy  as np # numpy is always useful
import os           # Loads the functions related to the operating system 

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection   import train_test_split

from lib_file import *

In [ ]:
data = # TASK: Load the data from 'training_scaled.csv'

## Step 1.2: Split features and targets

From the data, we should extract the features and targets:

In [ ]:
target_name   = 'tilted_radiation'
features_list = [ 'roof_tilt', 'roof_aspect', 'roof_area', 'roof_x', 'roof_y', 'aspect_sign', 'SIS',
                  'SISDIR', 'DHI', 'horizon_S', 'horizon_SSW', 'horizon_SWW', 'horizon_W',
                  'horizon_NWW', 'horizon_SSE', 'horizon_SEE', 'horizon_E', 'horizon_NEE',]

In [ ]:
y = ## TASK: EXTRACT THE TARGET COLUMN FROM data
X = ## TASK: EXTRACT THE FEATURES FROM data

## Step 1.3: Extract optimal number of features
In Tutorial 2, we have assessed the **optimal number of features** for modelling using a default KNN.
As the results in Tutorial 2 vary slightly due to the re-shuffling of the data, we will define it here as **6** for consistency. A set of 6 features should always yield some of the best results, even if not the very best.

**NOTE**: The optimal number of features can vary for different algorithms and hyper-parameter configurations. However, for the given exercise we will assume that the feature selection using the default KNN is representative for both KNN and SVR algorithms. For our dataset, this is approximately true.

In [ ]:
n_features = 6   # Define the optimal number of features

Now, we extract the variable `X_tuning` that contains the reduced dataset with only 6 features (based on the mutual regression criterion):

In [ ]:
mutual_info = ## TASK: INITIALIZE THE MUTUAL_INFO SELECTOR WITH n_features
X_tuning    = ## TASK: EXTRACT THE BEST FEATURES FROM X

## *Question*
- Which are the selectede best 6 features? (*HINT*: refer to Tutorial 2)

## Step 1.4: Split data into training and validation
For the initial exploration of the model parameters, we are mostly interested to look into *over-fitting* and *under-fitting* behaviour. For this, it is essential that we compare the *training* and *validation*-scores as we have done in Tutorial 1. We hence split the data into training and validation sets:

In [ ]:
X_train, X_val, y_train, y_val = # TASK: Split the data into 80% training and 20% validation data

**NOTE**: For cross-validation, we will not use the training and validation sets, but instead we use directly the `X_tuning` and `y`.

# Part 2 - KNN

The KNN regression algorithm is imported from the `sklean.neighbours` module and is documented here:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

## *Questions*:
- Which are the main hyper-parameters of KNN and what are their variable names in `scikit-learn`?
- What are the default values of these hyper-parameters?

## Step 2.1: Hyper-Parameter analysis

Usually, the default hyper-parameters are not those that provide the best results. To find the optimal hyper-parameters for a given dataset, we must **tune** the model. To develop a good intuition for the parameter choices during tuning, we need to understand *how* these impact the model performance. This is best assessed by comparing training and validation scores, which unveil some information about under-fitting and over-fitting behaviour. Hence, in this Step we will use the training and validation sets obtained in Step 1.4.

Here, we will focus on the most important hyper-parameter of KNN, the number of neighbours (`n_neighbors`, let's call it *k*), and its impact on the model performance. 
In Step 2.2, we will look at the systematic tuning of the algorithm and other hyperparameters.

When tuning an ML model, it is always useful to keep the default performance in mind. We can obtain the default performance as:

In [ ]:
knn = KNeighborsRegressor() # Initiate model
knn.fit(X_train, y_train)   # Fit model with training data

In [ ]:
print( 'Training score: %.3f' %knn.score(X_train, y_train) )
print( 'Validation score: %.3f' %knn.score(X_val, y_val) )

**_What if n_neighbours is very small?_**

The minimum *k* that we can choose is 1. So, let's train and score a KNN with 1 neighbor:

*HINT*: Hyper-parameters are always set at the *initialisation* of a model.

In [ ]:
knn = ## TASK: INITIATE MODEL WITH 1 NEIGHBOR
## TASK: FIT THE MODEL WITH THE TRAINING DATA.

And assess the model performance:

In [ ]:
## TASK: COMPUTE AND DISPLAY THE TRAINING SCORE
## TASK: COMPUTE AND DISPLAY THE VALIDATION SCORE

**_What if n_neighbours is very large?_**

The other extreme choice of *k* is a very large value. The maximum possible value is equal to the length of the training data (800 samples):

In [ ]:
knn = ## TASK: INITIATE MODEL WITH 800 NEIGHBORS
## TASK: FIT THE MODEL WITH THE TRAINING DATA.

In [ ]:
## TASK: COMPUTE AND DISPLAY THE TRAINING SCORE
## TASK: COMPUTE AND DISPLAY THE VALIDATION SCORE

## *Questions*
- a. Why is the KNN *training* score with 1 neighbour always 1, and with 800 neighbours always 0?
- b. How does the *validation* score compare to the *training* score for *k = 1* and *k = 800*?
- c. In which case is the model over-fitting, and where is it under-fitting?

## Step 2.2: Hyper-Parameter tuning
Testing every possible hyper-parameter individually, as we have done above, is clearly very time-consuming. To facilitate the process, we will use the `GridSearchCV` function of `scikit-learn`. This function allows to perform cross-validation for a defined set of values of one or several hyper-parameters, which has several advantages:

- It avoids the use of long codes for tuning (e.g. nested for-loops for tuning multiple parameters)
- It automatically stores all the cross-validation scores and computes the statistics
- It also logs the fitting and scoring times, which gives insight in the trade-off between computational time and performance
- It can run *in parallel*, which reduces the required time for testing large amounts of hyper-parameters
- It re-fits the estimator with the best performance on the entire training dataset, which can be used for testing (see Step 2.3)

In [ ]:
from sklearn.model_selection import GridSearchCV
import time     # For logging execution times

The `GridSearchCV` requires as input a default estimator and a dictionary with the hyper-parameter values to be tested. In our case, we have only one main parameter (`n_neighbors`). We can hence define a range of values for which to tune the estimator:

In [ ]:
hyperparams = { 'n_neighbors' : np.arange(1, 50) }  # Test values between 1 and 50

The GridSearchCV is performed by initialising a default model, declaring the GridSearchCV instance and fitting it on the *entire* feature set (performing the CV grid search). Additionally, we can log the time:

In [ ]:
tt = time.time()

knn = KNeighborsRegressor()                        # Initiate default KNN
knn_gridsearch = GridSearchCV(knn, hyperparams)    # Declare GridSearch instance
knn_gridsearch.fit(X_tuning, y)                    # Fit the grid-search (i.e. perform the grid-search)

print('Performed hyperparameter tuning in %.2fs' %(time.time()-tt))

##### Results of GridSearchCV
There are plenty of results produced by the GridSearchCV, but we are interested in three main ones:
- The average validation score across all folds
- The fitting time
- The scoring time

The results are saved in the `cv_results_` attribute of the gridsearch instance. We can visualise them by loading them into a dataframe:

In [ ]:
knn_gridsearch_results = pd.DataFrame(knn_gridsearch.cv_results_)
knn_gridsearch_results.head(3)

To see how the score changes with `n_neighbors`, we can look at the `mean_test_score` and the `std_test_score`, which shows the average validation score and its standard deviation across all k folds. The value of `n_neighbors` corresponding to each row is stored in the column `param_n_neighbors`. This data can be easily visualised, just as the number of features from Tutorial 2 (Step 3.2):

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (10,4))
knn_gridsearch_results.plot(x = 'param_n_neighbors', 
                            y = 'mean_test_score', 
                            yerr = 'std_test_score', 
                            ax = plt.gca())
plt.ylabel('Cross-validation score ($R^2$)')
plt.grid()
plt.tight_layout()

Additionally, the **best parameters** and the **best score** are saved as separate attributes of the grid search instance, which can be used to train the optimal model. Let's show these two attributes:

**NOTE**: The names of the attributes can be found in the documentation.

In [ ]:
# TASK: Show the best parameters of the GridSearchCV results

In [ ]:
# TASK: how the best score of the GridSearchCV results

## *Questions*
- a. What is the optimal number of neighbours for this problem? What is the best performance that can be achieved?
- b. The standard deviation is quite high. How can the standard deviation be reduced?
- c. [OPTIONAL]: What about other hyper-parameters? Does changing these (e.g. `weights`, `metric`) improve the model performance?

## Step 2.3: Model testing
Once we have completed the hyper-parameter tuning, we should always test the model performance on an independent **test set**. It is hereby important that the test data has not been used in the cross-validation process. In our example, the test data is taken from a different geographic area (see the Tutorial introduction document).
The test data is used for the final model evaluation and for comparison to other models, since it is not biased by the tuning process.

We have prepared a test dataset for you, which consists of two files:
- *test_data.csv*: The test dataset, where the feature transformation from Tutorial 2 has been applied
- *test_data_scaled.csv*: The above dataset, which has been standardized with respect to the training data

##### Load and transform test data
Since in this tutorial we work with scaled data, we should load the scaled test data and extract the features and target similar to the training data:

In [ ]:
test_data = pd.read_csv('test_data_scaled.csv', index_col = 0)

In [ ]:
X_test = test_data[ features_list ]
y_test = test_data[ target_name ]

However, we also applied the mutual information criterion above to select only the 6 best features. We can use the same `SelectKBest` instance (called `mutual_info` in Step 3.3, and which is already fitted to the training data) to transform the test dataset:

In [ ]:
X_test_reduced = mutual_info.transform( X_test )

##### Predict test labels
Now, we should predict the test labels for the best hyper-parameters found in the model tuning process. For this, the `GridSearchCV` has a very useful functionality, as it re-fits an estimator with the best found parameters. To obtain a prediction (or a score), we hence have to simply call the `predict()` function as we usually do:

In [ ]:
y_test_pred = knn_gridsearch.predict( X_test_reduced )

##### Evaluate the test prediction
To assess the test prediction, we can call the knn `score`-function, which we have also used during model tuning. However, as we have seen in Tutorial 1, the default $R^2$-score is not the only metric available to assess model performance. Usually, we try to compare different error metrics, in order to obtain a holistic overview of a model's performance. 

**TASK**: Compute the $R^2$-coefficient of determination and the MAE (mean absolute error) for the test set (refer to Tutorial 1).

##### Visualise the test prediction
Finally, we can also visualise the test results in the form of a scatterplot, with the "True" labels on the x-axis, and the "Predicted" labels on the y-axis. Ideally, the points should lie on a straight line of form y=x (this would correspond to an R2-value of 1).

**TASK**: Create a scatterplot of the test samples, with the targets on the x-axis and the predictions on the y-axis. Label the axes correctly and and a line of y=x to indicate the "perfect" prediction.

## *Questions*
- a. How does the test score compare to the validation score?
- b. How does the MAE compare to the MAE found for the linear regression in Tutorial 1?
- c. Do you notice any systematic bias in your scatterplot?

# Part 3 - Support Vector Regression (SVR)

The SVR algorithm is imported from the `sklean.svm` module and is documented here:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [ ]:
from sklearn.svm import SVR

## *Questions*:
- Which are the main hyper-parameters of SVM and what are their variable names in `scikit-learn`?
- What are the default values of these hyper-parameters?

## Step 3.1: Hyper-parameter analysis

Similar to the KNN, let's fit an SVR with default parameters:

**TASK**: initialise and fit a default SVR model and obtain the training and validation score.

The scores of the default SVR are not satisfactory, so we need to change the hyper-parameters. Here, we will look at two parameters - `C` and `gamma`, while keeping the default kernel and $\epsilon$. 

##### Varying C
Let's start by changing `C` - either to a very small value (0.001) or to a very big value (say, 10,000).

**TASK**: initialise and fit two SVR models (one with `C = 0.001` and one with `C = 10000`) and obtain the training and validation score.

##### Varying gamma
To test the performance of `gamma`, we should first understand the default value. Based on the documentation, we can compute `gamma` as: 

In [ ]:
gamma_default = 1 / (n_features * X_train.var())
gamma_default

Let's again check some very small (0.00001) and very high (10) values. As the default model performs badly, let's choose a value of `C` that lies somewhere in the expected range (e.g. C = 1000)

**TASK**: initialise and fit two SVR models (one with `gamma = 0.0001` and one with `gamma = 1`, and both with `C = 1000`) and obtain the training and validation scores.

## *Questions*
- a. How does the default model perform compared to the default KNN?
- b. For which values of `C` does the model over-fit, when does it under-fit?
- c. In which range do you expect the optimal value for `C`?
- d. In which "direction" (increasing, decreasing) does `gamma` lead to overfitting, when does it underfit?

## Step 3.2: Hyperparameter tuning

Tuning the hyper-parameters for the SVR is somewhat more cumbersome than for the KNN, for two reasons: 
1. We now have two main hyper-paramters to tune (`C` and `gamma`)
2. The model training is slower for the SVR, so the GridSearch requires more computational time.

To avoid searching a huge number of possible combinations, we can perform the GridSearchCV in an iterative fashion. First, we choose large ranges, which we decrease once we have identified the area where the model performs best to close in on the optimal values.

Let's start by choosing hyper-parameters in the orders of magnitude around the default values (remember that we already have some knowledge of the approximate range of the hyper-parameters from Step 3.1!):

In [ ]:
hyperparams = {
    'C'     : [1, 10, 100, 1000, 10000],      # Get orders of magnitude for C
    'gamma' : [0.0001, 0.001, 0.01, 0.1, 1]   # Get orders of magnitude for gamma
}

**NOTE**:  If you have a strong machine and have a lot of time, you can run many parameters at once, but often it is good to "test" first for the correct range of hyper-parameters. If you feel that the hyper-parameter tuning takes an excessive amount of time, consider stopping the notebook and reducing the range of the tested values.

With the dictionary of hyper-parameters and after declaring an instance of the default SVR model, we can call the `GridSearchCV` function and fit it with the features and the target.

In [ ]:
tt = time.time() # Log the current time

# TASK: Declare default SVR     
svr_gridsearch = # TASK: Declare an SVR grid-search with the hyperparameter dictionary
# TASK: Fit the SVR grid-search (i.e. perform the grid-search over all hyper-parameters)

print('Performed hyperparameter tuning in %.2fs' %(time.time()-tt))

In [ ]:
svr_gridsearch_results = # TASK: Obtain cv_results_ as a dataframe 

To see how the error changes with `C` and `gamma`, we can extract the `mean_test_score`. The data is best understandable if we expand it to two dimensions, one for C and one for gamma. For this, we can use panda's `pivot()` function:

In [ ]:
mean_val_score = svr_gridsearch_results.pivot(values = 'mean_test_score',   # Values to popoulate the 2D datagrame
                                              index = 'param_C',            # New index
                                              columns = 'param_gamma')      # New columns

Which we can then display as a formatted matrix:

In [ ]:
format_df( mean_val_score, cmap='YlGn', decimals = 3)

We can also look at the time for fitting and scoring:

In [ ]:
mean_fit_time = ## TASK: PIVOT THE svr_gridsearch_results TO EXTRACT THE mean_fit_time

In [ ]:
format_df( mean_fit_time, cmap='autumn_r', decimals = 3)

In [ ]:
mean_score_time = ## TASK: PIVOT THE svr_gridsearch_results TO EXTRACT THE mean_score_time
## TASK: DISPLAY FORMATTED MATRIX

**TASK**: Go back to the beginning of Step 3.2 and change the hyper-parameter values, closing in on the range of values with the highest scores. Iterate this process until you have found a satisfactory best estimator.

*HINT*: Don't forget about the *best parameter* and *best score* attributes that we have seen in Step 2.2.

## *Questions*
- a. In which range do you find the best hyper-parameters?
- b. Which process (fitting or scoring) is more time-intensive for the SVR?
- c. How do the fitting times compare to the performance? Do you notice a pattern? What does this suggest about the over/underfitting of the model?
- [OPTIONAL] d. Expand the model tuning by other hyper-parameters. What is the best score that you can achieve? What are the hyper-parameters to achieve this score? 

## Step 3.3: Evaluate the test set
Similar to the KNN, we should evaluate the prediction on the test set:

In [ ]:
y_test_pred_svr = # TASK: Predict the test labels for the best estimator found in step 3.2

**TASK**: Similar to Step 2.3, compute the R2 and MAE scores for the SVR on the test set, and create a scatterplot that shows both test predictions (KNN and SVR), as well as the line "y = x".

## *Questions*
- a. How does the test performance of the SVR compare to test performance of the KNN (R2 and MAE)?
- b. How to the scatterplots of the two test predictions compare?